# RAG
Retrieval Augmented Generation, 검색 증강 생성

LLM에게 외부 데이터를 context로 활용할 수 있도록 처리 하는 과정

## 사전처리 단계

1. 문서 불러오기(Loading)
2. 텍스트 나누기(Splitting)
3. 숫자로 바꾸기(Embedding)
4. 저장하기(VectorStore)

In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# 예시
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]  # 문서 불러오기 - 텍스트 나누기(1, 2)까지 완료된 상태

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# OpenAI의 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
embedding.embed_query('고양이보다는 개가 더 좋지')

# Vector Store에 임베딩 후 저장(3, 4) - 메모리에
vectorstore = Chroma.from_documents(documents, embedding=embedding)

In [6]:
vectorstore.similarity_search(
    '고양이보다 개가 더 좋아', 
    k=2
)

[Document(id='2048dff6-c685-4321-aee2-599600359c6a', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='387917e9-8ec1-4d18-b27e-d904ee4239b5', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [18]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('../data/spri.pdf')
docs = loader.load()

print(len(docs))

# for doc in docs:
#     print(doc)

23


In [19]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Recursive~: 대충 자르는 splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 글자 수
    chunk_overlap=50  # 두 청크 사이에 겹치는 글자 수를 50개
)

docs_splitted = text_splitter.split_documents(docs)
print(len(docs_splitted))

# for doc in docs_splitted:
#     print(doc)
#     print('========================================================')

72


In [ ]:
# 3. 벡터 임베딩 + vectorstore 저장
from langchain_openai import OpenAIEmbeddings  # 3. 임베딩 담당
from langchain_community.vectorstores import FAISS  # 4. 벡터스토어 담당

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=docs_splitted, embedding=embedding)

vectorstore.similarity_search(
    '미국 대통령과 관련된 문서들 가져와봐'
)

[Document(id='e1df93b2-8c5a-4acb-9f87-62822783df02', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': '../data/spri.pdf', 'file_path': '../data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Do

## 검색 증강 단계

1. 사용자 질문(Query)
2. 검색(Retrieve)
3. LLM
4. 최종 답변

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트(langchain hub에서)
prompt = hub.pull('rlm/rag-prompt')

# LLM(gpt)
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기(Retriever)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}  # 들어온 인자가 그대로 들어옴
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식을 다 가져와줘')

'삼성전자는 자체 개발한 생성 AI인 ‘삼성 가우스’를 공개했으며, 언어, 코드, 이미지 모델로 구성되어 온디바이스에서 작동합니다. 이 AI는 외부 유출 위험이 낮고, 스마트폰 등 다양한 제품에 단계적으로 탑재될 예정입니다. 2023년 11월 ‘삼성 AI 포럼 2023’에서 최초 공개되었으며, 향후 경쟁 제품들과의 차별화를 기대할 수 있습니다.'

In [ ]:
# 관심있는 데이터 소스로 RAG 해보기
